# No 2

In [6]:
import pandas as pd
import math
from scipy import stats
from scipy.stats import t


data_path = 'weather.csv'
data = pd.read_csv(data_path)

def deteksi_outlier_iqr_manual(data):
    outlier_indices = {}
    for kolom in data.select_dtypes(include=['number']).columns:
        Q1 = data[kolom].quantile(0.25)
        Q3 = data[kolom].quantile(0.75)
        IQR = Q3 - Q1
        batas_bawah = Q1 - 1.5 * IQR
        batas_atas = Q3 + 1.5 * IQR
        outlier_mask = (data[kolom] < batas_bawah) | (data[kolom] > batas_atas)
        outlier_indices[kolom] = data[kolom][outlier_mask].index.tolist()
    return outlier_indices

def imputasi_outlier(data, outlier_indices):
    for kolom, indices in outlier_indices.items():
        median = data[kolom].median()
        data.loc[indices, kolom] = median
    return data

outlier_indices_manual = deteksi_outlier_iqr_manual(data)

data_imputasi = imputasi_outlier(data.copy(), outlier_indices_manual)


#Belah dua data
length = data_imputasi.shape[0]
areaA = data_imputasi[:1000]
areaB = data_imputasi[1000:]
print(areaA.describe())


        Unnamed: 0  Temperature     Humidity  Precipitation   Wind_Speed  \
count  1000.000000  1000.000000  1000.000000    1000.000000  1000.000000   
mean    499.500000    24.949983    70.035900       9.981231    14.751966   
std     288.819436     2.919950     9.623194       4.660435     4.940096   
min       0.000000    17.100577    44.860207      -2.889661     2.230926   
25%     249.750000    22.983713    63.113037       6.853253    11.335045   
50%     499.500000    24.894164    70.011814       9.908852    14.702498   
75%     749.250000    26.986035    76.621300      13.059373    18.178072   
max     999.000000    32.830039    96.747550      23.013390    27.770656   

       Cloud_Coverage     Pressure     UV_Index  Air_Quality   Visibility  
count     1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  
mean        49.882556   999.705214     5.443798    66.021552    15.106431  
std          9.469565    57.235568     2.597302     8.351779     4.695736  
min        

In [12]:
# Soal 6.1
# 1. Menentukan Hipotesis Nol
# H0: u1 = u2

# 2. Menentukan Hipotesis Alternatif
# H1: u1 != u2

# 3. Menentukan tingkat signifikan a
alpha = 0.05

# 4. Menentukan uji statistik dan daerah kritis
# Sebab kedua data yang telah dipecah berasal dari satu tempat yang sama
# Asumsikan standar deviasi uniform
# Akan digunakan distribusi t agar lebih akurat
# Sebab berdasarkan nomor 4, bentuk distribusi data tidak benar benar normal
critical_value_low = stats.t.ppf(alpha/2, 1000 + 1000 -2) #0.025
critical_value_high = stats.t.ppf(1 - alpha/2, 1000 + 1000 -2) #0.975

# 5. Hitung p-value
# 5.1. Hitung mean dan deviasi untuk A
mean_areaA = sum(areaA['Humidity']/1000)

varians_areaA = 0
for humidity in areaA['Humidity']:
    varians_areaA += math.pow((humidity - mean_areaA), 2)
varians_areaA /= 1000 - 1
standardeviasiA = math.sqrt(varians_areaA)

# 5.2. Hitung mean dan deviasi untuk B
mean_areaB = sum(areaB['Humidity']/1000)

varians_areaB = 0
for humidity in areaB['Humidity']:
    varians_areaB += math.pow((humidity - mean_areaB), 2)
varians_areaB /= 1000 - 1
standardeviasiB = math.sqrt(varians_areaB)

# 5.3. Hitung sp
sp2 = ((1000 - 1) * math.pow(standardeviasiA, 2) + (1000 - 1) * math.pow(standardeviasiB, 2)) / (1000 + 1000 - 2)
sp = math.sqrt(sp2)

# 5.4 Hitung
t_value = ((mean_areaA - mean_areaB) - 0) / (sp * math.sqrt(1/1000 + 1/1000))

# 5.5 Hitung degree of freedom
v = 1000 + 1000 - 2

# 5.6 Hitung p value
p_value = 2 * (1 - stats.t.cdf(abs(t_value), df=v))

# 6. Ambil keputusan
print(f"t-value: {t_value}")
print(f"p-value: {p_value}")
print(f"Critical value (low): {critical_value_low}")
print(f"Critical value (high): {critical_value_high}")

# Compare t-value with critical values to make a decision
if t_value < critical_value_low or t_value > critical_value_high:
    print(f"\n{p_value} < {alpha}")
    print("Tolak H0")
    print("Rata-rata humidity area A berbeda dengan rata-rata humidity area B")
else:
    print(f"\n{p_value} > {alpha}")
    print("Gagal menolak H0")
    print("Rata-rata humidity area A sama dengan rata-rata humidity area B")



t-value: -0.12659222179898152
p-value: 0.899275891151988
Critical value (low): -1.961152014836706
Critical value (high): 1.9611520148367056

0.899275891151988 > 0.05
Gagal menolak H0
Rata-rata humidity area A sama dengan rata-rata humidity area B


In [14]:
# Soal 6.2
# 1. Menentukan Hipotesis Nol
# H0: u1 - u2 = 5

# 2. Menentukan Hipotesis Alternatif
# H1: u1 - u2 != 5

# 3. Menentukan tingkat signifikan a
alpha = 0.05

# 4. Menentukan uji statistik dan daerah kritis
# Sebab kedua data yang telah dipecah berasal dari satu tempat yang sama
# Asumsikan standar deviasi uniform
# Akan digunakan distribusi t agar lebih akurat
# Sebab berdasarkan nomor 4, bentuk distribusi data tidak benar benar normal
critical_value_low = stats.t.ppf(alpha/2, 1000 + 1000 -2) #0.025
critical_value_high = stats.t.ppf(1 - alpha/2, 1000 + 1000 -2) #0.975

# 5. Hitung p-value
# 5.1. Hitung mean dan deviasi untuk A
mean_areaA = sum(areaA['Wind_Speed']/1000)

varians_areaA = 0
for wind in areaA['Wind_Speed']:
    varians_areaA += math.pow((wind - mean_areaA), 2)
varians_areaA /= 1000 - 1
standardeviasiA = math.sqrt(varians_areaA)

# 5.2. Hitung mean dan deviasi untuk B
mean_areaB = sum(areaB['Wind_Speed']/1000)

varians_areaB = 0
for wind in areaB['Wind_Speed']:
    varians_areaB += math.pow((wind - mean_areaB), 2)
varians_areaB /= 1000 - 1
standardeviasiB = math.sqrt(varians_areaB)

# 5.3. Hitung sp
# sp2 = ((n1 - 1) * math.pow(s1, 2) + (n2 - 1) * math.pow(s2, 2)) / (n1 + n2 - 2)
sp2 = ((1000 - 1) * math.pow(standardeviasiA, 2) + (1000 - 1) * math.pow(standardeviasiB, 2)) / (1000 + 1000 - 2)
sp = math.sqrt(sp2)

# 5.4 Hitung
# t_value = ((u1 - u2) - d0) / (sp * math.sqrt(1/n1 + 1/n2))
t_value = ((mean_areaA - mean_areaB) - 5) / (sp * math.sqrt(1/1000 + 1/1000))

# 5.5 Hitung degree of freedom
v = 1000 + 1000 - 2

# 5.6 Hitung p value
p_value = 2 * (1 - stats.t.cdf(abs(t_value), df=v))

# 6. Ambil keputusan
print(f"t-value: {t_value}")
print(f"p-value: {p_value}")
print(f"Critical value (low): {critical_value_low}")
print(f"Critical value (high): {critical_value_high}")

# Compare t-value with critical values to make a decision
if t_value < critical_value_low or t_value > critical_value_high:
    print(f"\n{p_value} < {alpha}")
    print("Tolak H0")
    print("Wind Speed area A tidak lebih tinggi 5mm dari area B")
else:
    print(f"\n{p_value} > {alpha}")
    print("Gagal menolak H0")
    print("Wind Speed area A lebih tinggi 5mm dari area B")


t-value: -24.98798337104825
p-value: 0.0
Critical value (low): -1.961152014836706
Critical value (high): 1.9611520148367056

0.0 < 0.05
Tolak H0
Wind Speed area A tidak lebih tinggi 5mm dari area B
        Unnamed: 0  Temperature     Humidity  Precipitation   Wind_Speed  \
count  1000.000000  1000.000000  1000.000000    1000.000000  1000.000000   
mean    499.500000    24.949983    70.035900       9.981231    14.751966   
std     288.819436     2.919950     9.623194       4.660435     4.940096   
min       0.000000    17.100577    44.860207      -2.889661     2.230926   
25%     249.750000    22.983713    63.113037       6.853253    11.335045   
50%     499.500000    24.894164    70.011814       9.908852    14.702498   
75%     749.250000    26.986035    76.621300      13.059373    18.178072   
max     999.000000    32.830039    96.747550      23.013390    27.770656   

       Cloud_Coverage     Pressure     UV_Index  Air_Quality   Visibility  
count     1000.000000  1000.000000  1000.

In [16]:
# Soal 6.3
# 1. Menentukan Hipotesis Nol
# H0: σ1^2 = σ2^2

# 2. Menentukan Hipotesis Alternatif
# H1: σ1^2 != σ2^2

# 3. Menentukan tingkat signifikan a
alpha = 0.05

# 4. Menentukan uji statistik dan daerah kritis
# digunakan distribusi f
critical_value_low = stats.f.ppf(alpha/2, 1000 - 1, 1000 - 1) #0.025
critical_value_high = stats.f.ppf(1 - alpha/2, 1000 - 1, 1000 - 1) #0.975

# 5. Hitung p-value
# 5.1. Hitung mean dan deviasi untuk A
mean_areaA = sum(areaA['Air_Quality']/1000)

varians_areaA = 0
for wind in areaA['Air_Quality']:
    varians_areaA += math.pow((wind - mean_areaA), 2)
varians_areaA /= 1000 - 1
standardeviasiA = math.sqrt(varians_areaA)

# 5.2. Hitung mean dan deviasi untuk B
mean_areaB = sum(areaB['Air_Quality']/1000)

varians_areaB = 0
for wind in areaB['Air_Quality']:
    varians_areaB += math.pow((wind - mean_areaB), 2)
varians_areaB /= 1000 - 1
standardeviasiB = math.sqrt(varians_areaB)

# 5.3. Hitung f
f = math.pow(standardeviasiA, 2) / math.pow(standardeviasiB, 2)

# 5.4 Hitung degree of freedom
v1 = 1000 - 1
v2 = 1000 - 1

# 5.5 Hitung p value
p_value = 2 * (1 - stats.f.cdf(abs(t_value), v1, v2))

# 6. Ambil keputusan
print(f"f-value: {f}")
print(f"p-value: {p_value}")
print(f"Critical value (low): {critical_value_low}")
print(f"Critical value (high): {critical_value_high}")

# Compare t-value with critical values to make a decision
if f < critical_value_low or f > critical_value_high:
    print(f"\n{p_value} < {alpha}")
    print("Tolak H0")
    print("Variansi Air Quality daerah A tidak sama dengan B")
else:
    print(f"\n{p_value} > {alpha}")
    print("Gagal menolak H0")
    print("Variansi Air Quality daerah A sama dengan B")

f-value: 1.2447171740304441
p-value: 2.220446049250313e-16
Critical value (low): 0.8832987168478774
Critical value (high): 1.1321198377470543

2.220446049250313e-16 < 0.05
Tolak H0
Variansi Air Quality daerah A tidak sama dengan B


In [37]:
# Soal 6.4
# 1. Menentukan Hipotesis Nol
# H0: p1 = p2

# 2. Menentukan Hipotesis Alternatif
# H1: p1 > p2

# 3. Menentukan tingkat signifikan a
alpha = 0.05

# 4. Menentukan uji statistik dan daerah kritis
# Digunakan distribusi Z

# Melakukan filter pada area A
filter = areaA['Precipitation'] < 7
prepA = areaA[filter]

# Melakukan filter pada area B
filter = areaB['Precipitation'] < 7
prepB = areaB[filter]

# Hitung proporsi
p1 = len(prepA)/1000
p2 = len(prepB)/1000

# Hitung proporsi total
p = (len(prepA) + len(prepB))/2000
q = 1 - p

# 5. Hitung p-value
# Hitung z
countA = len(prepA['Precipitation'])
countB = len(prepB['Precipitation'])

z = (p1 - p2)/math.sqrt(p * q *((1/countA) + (1/countB)))

# Hitung p
p_val = round(1 - stats.norm.cdf(z))

# Hitung critical value
critical_value = stats.norm.ppf(alpha)

# 6. Ambil keputusan
print(f"z-value: {f}")
print(f"p-value: {p_value}")
print(f"Critical value: {critical_value}")

# Compare t-value with critical values to make a decision
if z < critical_value:
    print(f"\n{p_val} < {alpha}")
    print("Tolak H0")
    print("Proporsi Precipitation yang kurang dari 7 pada daerah A tidak sama dengan proporsi di daerah B")
else:
    print(f"\n{p_val} > {alpha}")
    print("Gagal menolak H0")
    print("Proporsi Precipitation yang kurang dari 7 pada daerah A sama dengan proporsi di daerah B")

z-value: 1.2447171740304441
p-value: 2.220446049250313e-16
Critical value: -1.6448536269514729

1 > 0.05
Gagal menolak H0
Proporsi Precipitation yang kurang dari 7 pada daerah A sama dengan proporsi di daerah B
